In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import inspect
import seaborn as sns 
import matplotlib.pyplot as plt
#from datetime import timedelta

def list_columns(df):
    for col in df: 
        print(col, end='   ')

def row_count(df):
    return len(df)

In [2]:
df1 = pd.read_csv('Shekar-Dataset-1/last_week.csv', index_col=0) 
df2 = pd.read_csv('Shekar-Dataset-1/station_info.csv', index_col=0)



#data = [lw, si]

#for df in data: # Change the coloumn names to something I feel more used to. 
#    df.rename(columns={"dteday": "date", "cnt": "total_rentals", 'casual': 'casual_rentals', 'registered': 'registered_rentals', 'mnth':'month'}, inplace=True)

In [3]:
df1.describe()

,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
count,52943.000000,52512.000000,57934.000000,57934.000000,57848.000000,57848.000000
mean,31359.690233,31360.699612,38.903398,-77.031929,38.902358,-77.031565
std,252.234932,252.048691,0.027371,0.031668,0.027130,0.031517
min,31000.000000,31000.000000,38.782633,-77.368416,38.780000,-77.368447
25%,31212.000000,31213.000000,38.890496,-77.045128,38.890000,-77.045000
50%,31272.000000,31271.000000,38.902674,-77.032008,38.902386,-77.031887
75%,31603.000000,31603.000000,38.915417,-77.015970,38.912717,-77.015861
max,32609.000000,32901.000000,39.123513,-76.820000,39.123513,-76.820000


In [14]:
print(df1['start_station_name'].value_counts)

<bound method IndexOpsMixin.value_counts of ride_id
B3FA23181ADC06FE                       Georgia Ave & Spring St
6A153F2AD6700EE1    Merrifield Cinema & Merrifield Town Center
7265E4C759F1EDC0                      Columbia & Ontario Rd NW
18FB278BE5F2B0C7                                 1st & H St NW
294E78FE02E1C65F                                 5th & K St NW
                                       ...                    
A964858139D8693F                 17th St & Independence Ave SW
C027BB4E110C06B7                 17th St & Independence Ave SW
3EF5B216B64E7FDD                 17th St & Independence Ave SW
C37E11998FDC1078                 17th St & Independence Ave SW
56B80D92A24B8C93                 17th St & Independence Ave SW
Name: start_station_name, Length: 50369, dtype: object>


In [5]:
# https://www.semicolonworld.com/question/54955/mapping-columns-from-one-dataframe-to-another-to-create-a-new-column
df1['capacity_start_station'] = df1.start_station_name.map(dict(df2[['Name', 'Capacity']].values))
df1['capacity_end_station'] = df1.end_station_name.map(dict(df2[['Name', 'Capacity']].values))

df1['started_at'] = pd.to_datetime(df1['started_at'], format='%Y-%m-%d %H:%M:%S')# <-- How to change the datetime format to '%d.%m.%Y'| Use in a new line of code: dt.strftime('%m/%d/%Y')
df1['ended_at'] = pd.to_datetime(df1['ended_at'], format='%Y-%m-%d %H:%M:%S')
df1['day'] = pd.to_datetime(df1['ended_at'], format='%Y-%m-%d')
df1['rental_duration'] = df1['ended_at']-df1['started_at']
df1['rental_duration_num'] = df1['ended_at']-df1['started_at']
df1['rental_duration_num_min'] = df1['rental_duration_num'].dt.seconds//60
df1['station_pair'] = df1['start_station_id'].astype(str) + df1['end_station_id'].astype(str)
df1['combined_capacity'] = df1['capacity_start_station']*df1['capacity_end_station']


In [6]:
total = df1.isnull().sum().sort_values(ascending=False)
percent_1 = df1.isnull().sum()/df1.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

,Total,%
combined_capacity,7565,13.1
capacity_end_station,5555,9.6
end_station_name,5422,9.4
end_station_id,5422,9.4
capacity_start_station,5150,8.9


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57934 entries, B3FA23181ADC06FE to 56B80D92A24B8C93
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   rideable_type            57934 non-null  object         
 1   started_at               57934 non-null  datetime64[ns] 
 2   ended_at                 57934 non-null  datetime64[ns] 
 3   start_station_name       52943 non-null  object         
 4   start_station_id         52943 non-null  float64        
 5   end_station_name         52512 non-null  object         
 6   end_station_id           52512 non-null  float64        
 7   start_lat                57934 non-null  float64        
 8   start_lng                57934 non-null  float64        
 9   end_lat                  57848 non-null  float64        
 10  end_lng                  57848 non-null  float64        
 11  member_casual            57934 non-null  object         
 1

In [8]:
df1_raw = df1
df1 = df1.dropna()


In [9]:
print(((row_count(df1_raw)-row_count(df1))/row_count(df1_raw))*100,'% of the inital data has been droped!')

13.057962509062035 % of the inital data has been droped!


In [10]:
list_columns(df1)

rideable_type   started_at   ended_at   start_station_name   start_station_id   end_station_name   end_station_id   start_lat   start_lng   end_lat   end_lng   member_casual   day   capacity_start_station   capacity_end_station   rental_duration   rental_duration_num   rental_duration_num_min   station_pair   combined_capacity   

In [11]:
print(len(df1.index))
print(len(df1_raw.index))

50369
57934


In [12]:
df = df1 # Chage to use raw data.

## Plotting

### Comparting the top staition paris total ride duration againt the overal total ride duration.

In [13]:
overall_total_ride_duration_in_years = sum(data['rental_duration_num_min']/60/24/365)
print(overall_total_ride_duration_in_years, 'Years')

NameError: name 'data' is not defined

### Capacity utilization of the stations during the year. & used by member or casual. 

In [ ]:
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
list_columns(df)

In [ ]:
df_weekly = df.resample("D", on="started_at").mean()
df_weekly.head()

df.set_index("started_at").rolling(3).mean().head()

In [ ]:
df['rental_duration'].plot(linewidth=1.5);


### Create a flow map